# Practical 10: Applying the Crank-Nicolson method in the Black-Scholes

model

Computational Finance with Python

[Alet Roux](https://www.york.ac.uk/maths/staff/alet-roux/) ([Department
of Mathematics](https://maths.york.ac.uk), University of York)

Click on the following to open this file in Google Colab:

<figure>
<a
href="https://colab.research.google.com/github/aletroux/comp-finance-python/blob/main/practicals/P10_Crank-Nicolson_Black-Scholes_prac.ipynb"><img
src="https://colab.research.google.com/assets/colab-badge.svg"
alt="Open In Colab" /></a>
<figcaption>Open In Colab</figcaption>
</figure>

The aim of this practical is to explore the Crank-Nicolson method in the
context of the Black-Scholes model. The focus in this practical is on
stability and the greeks.

# 1. Option pricing

The code in the following cell performs a finite difference
approximation for pricing a European put option with strike $K=90$ and
maturity date $T=1$ using the Crank-Nicolson method in the Black-Scholes
model with parameters $S_0=100$, $r=0.05$ and $\sigma=0.2$. In this case
the partial differential equation and initial value is $$\begin{aligned}
\frac{\partial V}{\partial \tau}(\tau,S) &= r S\frac{\partial V}{\partial S}(\tau,S) + \tfrac{1}{2}\sigma^2 S^2\frac{\partial^2 V}{\partial S^2}(\tau,S)  -r V(\tau,S), \\
V(0, S) &= (K-S)^+ \text{ for all }S\in[0,\infty).
\end{aligned}
$$ We choose the boundary conditions $$\begin{aligned}
\lim_{S\rightarrow 0} \frac{\partial^2 V}{\partial S^2}(\tau,S) &= 0, \\
\lim_{S\rightarrow \infty} V(\tau, S) &= 0.
\end{aligned}
$$

Study the code carefully to see how it works.

In [1]:
import numpy as np
import math
import scipy.sparse as ssparse

def CN_price_put (Smin, Smax, T, K, r, sigma, M, N):    
    """Finite difference approximation of put option price in Black-Scholes model
    using Crank-Nicolson method.
    Arguments:
        Smin: lower boundary of grid
        Smax: upper boundary of grid
        T: maturity date
        K: strike
        r: interest rate
        r: constant volatility
        M: number of steps in direction tau
        N: number of steps in direction S
    Returns:
        V: matrix of results, V[i,j] is value at 
            iT/M and Smin + j(Smax-Smin)/N
        tau: vector of time steps
        S: vector of stock prices
    """

    V = np.zeros((M+1,N+1))

    # discretization parameters
    tau = np.linspace(0, T, M+1)
    S = np.linspace(0, Smax, N+1)
    Sint = S[1:-1]    
    dtau = tau[1]-tau[0]
    dS = S[1] - S[0]
    
    #initial value
    V[0] = np.maximum(K - S,0)

    # parameters that don't depend on time step
    alpha = dtau/2/dS**2 * sigma**2 * Sint**2
    beta = r*dtau/2/dS*Sint

    # Backward Euler matrix and vector
    uB = - alpha - beta
    dB = 2*alpha + r*dtau + 1
    lB = beta - alpha
    #impose boundary conditions
    uB[0] -= lB[0]
    dB[0] += 2*lB[0]
    #matrix
    AB = ssparse.diags(diagonals=[1+dB, lB[1:], uB[:-1]], offsets=[0, -1, 1], 
                       shape=(N-1, N-1), format='csc')
    ABsolver = ssparse.linalg.factorized(AB)

    # Forward Euler matrix and vector
    uF = alpha + beta
    dF = 1 - r*dtau- 2*alpha
    lF = alpha - beta
    #impose boundary conditions
    uF[0] -= lF[0]
    dF[0] += 2*lF[0]
    #matrix
    AF = ssparse.diags(diagonals=[1+dF, lF[1:], uF[:-1]], offsets=[0, -1, 1], 
                       shape=(N-1, N-1), format='csc')

    for i in range(M):
        V[i+1, 1:-1] = ABsolver(AF @ V[i, 1:-1])

        #impose boundary conditions
        V[i+1,0] = -V[i+1,2] + 2*V[i+1,1]

    return V, tau, S
                    
# Strike
K = 90

# Black-Scholes parameters
T = 1
S0 = 100
r = 0.05
sigma = 0.2

# FD parameters
Smin = 0
Smax = 2*S0

M = 120
N = 120

V, tau, S = CN_price_put (Smin, Smax, T, K, r, sigma, M, N)


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

fig = plt.figure(figsize = (8,6), constrained_layout=True)
ax = plt.axes(projection='3d')
ax.view_init(elev=20, azim=135)

ax.set_xlabel("Time to maturity $\\tau$")
ax.set(xlim = [tau[0],tau[-1]])
ax.tick_params(axis='x', pad=0)
ax.xaxis.pane.fill = False

ax.set_ylabel("Stock price $S$")              
ax.set(ylim = [S[0],S[-1]])
ax.tick_params(axis='y', pad=0)
ax.yaxis.pane.fill = False

ax.set_zlabel("$V(\tau,S)$")              
ax.zaxis.pane.fill = False

Ss, taus = np.meshgrid(S, tau)

ax.set(title = f"Crank-Nicolson approximation for put option with strike $K={K}$")

surf = ax.plot_surface(taus, Ss, V, rstride=1, cstride=1, linewidth=0, cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5);

In [3]:
fig, ax = plt.subplots(figsize = (12,7))

# approximation
ax.plot(S, V[-1], marker = '.')

# plot cosmetics
ax.set(title = f"Crank-Nicolson approximation at time to maturity $\\tau=T$ for put option "
       f"with strike $K={K}$")
ax.set(xlabel="Stock price $S$", xlim=(Smin, Smax))
ax.xaxis.grid(True)
ax.yaxis.grid(True)

# 2. Stability

The Crank-Nicolson method is stable if the eigenvalues of the matrix
$$A = (I + A^{\text{BE}})^{-1}(I + A^{\text{FE}})$$ are less than 1 in
absolute value.

It is not practical to incorporate a check for stability into the finite
difference procedure (it would slow it down); however when (as here)
this matrix is independent of the time step, then can be created
separately from the finite difference procedure in order to calculate
its eigenvalues.

<span class="theorem-title">**Exercise 1**</span> Write code in Python
to create the matrix $A$ connected with the boundary conditions above,
and determine the absolute value of its largest and smallest
eigenvalues.

In [4]:
#Insert code here

# 3. Greeks

Once the finite difference approximations have been calculated, we can
use them to approximate the greeks. The **delta** is given as
$$\Delta_{i,j}= 
\begin{cases}
\frac{1}{\Delta S}(V_{i,1} - V_{i,0}) & \text{if }j=0,\\
\frac{1}{2\Delta S}(V_{i,j+1} - V_{i,j-1}) & \text{if }j=1,\ldots,N-1,\\
\frac{1}{\Delta S}(V_{i,N} - V_{i,N-1}) & \text{if }j=N.
\end{cases}
$$ and the **gamma** as $$\Gamma_{i,j}= 
\begin{cases}
\Gamma_{i,1} & \text{if }j=0,\\
\frac{1}{\Delta S^2}(V_{i,j+1} - 2V_{i,j} + V_{i,j-1}) & \text{if }j=1,\ldots,N-1,\\
\Gamma_{i,N-1} & \text{if }j=N.
\end{cases}
$$

<span class="theorem-title">**Exercise 2**</span> Approximate the value
of the gamma and delta at all the grid points used for the finite
difference approximation of the put option price.

Present your results graphically.

In [6]:
#Insert code here

You can now start with Assignment 5.